In [1]:
import numpy as np
import pandas as pd
from fbprophet import Prophet
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform
import copy
#import warnings
#warnings.filterwarnings('ignore')

# Introduction

The basis of the model will be the Facebook Prophet package. But, the first thing we need to do is load the preprocessed data.


In [2]:
df = pd.read_csv("data/preprocessed.csv")

In [3]:
df = df[df.columns[1:]]
df.head()

,ds,y
0,2015-07-01 05:00:00,162827.0
1,2015-07-01 06:00:00,335153.0
2,2015-07-01 07:00:00,333837.0
3,2015-07-01 08:00:00,398386.0
4,2015-07-01 09:00:00,388954.0


Now we run the model with a 75-25 train test split.

In [4]:
spoint = int(df.shape[0]*0.75)
train_df = df.iloc[:spoint]
test_df = df.iloc[spoint:]
#Check split since I did it manually
train_df.shape[0] + test_df.shape[0] == df.shape[0]

True

Now we train the basic models

In [5]:
model = Prophet()
model.fit(train_df)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [6]:
eval_df = model.predict(pd.DataFrame(train_df['ds']))
eval_df.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2015-07-01 05:00:00,432089.920648,424866.476908,516416.892919,432089.920648,432089.920648,38640.777508,38640.777508,38640.777508,-25279.942936,...,11005.316492,11005.316492,11005.316492,52915.403951,52915.403951,52915.403951,0.0,0.0,0.0,470730.698156
1,2015-07-01 06:00:00,432093.576126,402987.726504,496869.422091,432093.576126,432093.576126,16304.402767,16304.402767,16304.402767,-47721.157660,...,11030.513571,11030.513571,11030.513571,52995.046855,52995.046855,52995.046855,0.0,0.0,0.0,448397.978892
2,2015-07-01 07:00:00,432097.231603,384138.387691,477432.310472,432097.231603,432097.231603,-763.670208,-763.670208,-763.670208,-64916.221782,...,11077.716076,11077.716076,11077.716076,53074.835497,53074.835497,53074.835497,0.0,0.0,0.0,431333.561394
3,2015-07-01 08:00:00,432100.887080,369489.171976,467033.448455,432100.887080,432100.887080,-11441.822973,-11441.822973,-11441.822973,-75742.065391,...,11145.472993,11145.472993,11145.472993,53154.769424,53154.769424,53154.769424,0.0,0.0,0.0,420659.064106
4,2015-07-01 09:00:00,432104.542557,368381.786989,462543.527737,432104.542557,432104.542557,-14906.350065,-14906.350065,-14906.350065,-79373.208471,...,11232.010233,11232.010233,11232.010233,53234.848173,53234.848173,53234.848173,0.0,0.0,0.0,417198.192492


In [7]:
print(metrics.r2_score(train_df['y'], model.predict(pd.DataFrame(train_df['ds'])).yhat), 
      metrics.r2_score(test_df['y'],model.predict(pd.DataFrame(test_df['ds'])).yhat), 
      metrics.r2_score(df['y'], model.predict(pd.DataFrame(df['ds'])).yhat))

0.7824039517999025 0.687577002082496 0.7638128022011801


As expected, the model does the best on the training set with noticably worse performance on the test set. Prophet is performing worse than the random forest model from the EDA, so we are going to combine it with the random forest model to improve the accuracy when compared against either.

In [8]:
rfr = RandomForestRegressor()

In [9]:
sdf = pd.DataFrame(df)
sdf['sy'] = sdf.y.shift(-24)
sdf['ssy'] = sdf.y.shift(-48)
sdf = sdf.iloc[:-48]
sdf.head(25)

,ds,y,sy,ssy
0,2015-07-01 05:00:00,162827.0,453284.0,440084.0
1,2015-07-01 06:00:00,335153.0,429199.0,415203.0
2,2015-07-01 07:00:00,333837.0,407007.0,394227.0
3,2015-07-01 08:00:00,398386.0,395194.0,383147.0
4,2015-07-01 09:00:00,388954.0,387654.0,373941.0
5,2015-07-01 10:00:00,392487.0,390157.0,371552.0
6,2015-07-01 11:00:00,404647.0,401643.0,373312.0
7,2015-07-01 12:00:00,422227.0,417369.0,380160.0
8,2015-07-01 13:00:00,442131.0,437741.0,396051.0
9,2015-07-01 14:00:00,464371.0,459562.0,418789.0


In [23]:
spoint = int(0.75*sdf.shape[0])
X_train, X_test, y_train, y_test = sdf[['y', 'sy']].iloc[:spoint], sdf[['y', 'sy']].iloc[spoint:], sdf.ssy.iloc[:spoint], sdf.ssy.iloc[spoint:]
X_train.shape, y_test.shape
X_train.head()

,y,sy
0,162827.0,453284.0
1,335153.0,429199.0
2,333837.0,407007.0
3,398386.0,395194.0
4,388954.0,387654.0


In [11]:
rfr.fit(np.array(X_train), y_train)


RandomForestRegressor()

In [12]:
#use the scores on the trainging data set to determine weights for adding the models together
rf_score = metrics.r2_score(y_train, np.array(rfr.predict(np.array(X_train))))
p_score = metrics.r2_score(train_df['y'],model.predict(pd.DataFrame(train_df['ds'])).yhat)

In [29]:
score_sum = rf_score+p_score
def combine_predict(X):
    return (p_score*model.predict(pd.DataFrame(X['ds'])).yhat + rf_score*rfr.predict(X[['y', 'sy']]))/score_sum


In [30]:
#prep_df = pd.DataFrame()
ds_train, ds_test = sdf.ds.iloc[:int(0.75*sdf.shape[0])], sdf.ds.iloc[int(0.75*sdf.shape[0]):]
y_test = sdf.sy.iloc[int(0.75*sdf.shape[0]):]
prep_df = pd.concat([pd.DataFrame(ds_test), X_test], axis=1)

In [31]:
prep_df.head()

,ds,y,sy
36928,2019-09-16 21:00:00,629377.0,610432.0
36929,2019-09-16 22:00:00,632197.0,613776.0
36930,2019-09-16 23:00:00,623839.0,605297.0
36931,2019-09-17 00:00:00,608889.0,591760.0
36932,2019-09-17 01:00:00,593786.0,578988.0


In [32]:
pred_df = combine_predict(prep_df)

In [33]:
comb_score = metrics.r2_score(y_test, pred_df)

In [36]:
X_test.head()

,y,sy
36928,629377.0,610432.0
36929,632197.0,613776.0
36930,623839.0,605297.0
36931,608889.0,591760.0
36932,593786.0,578988.0


In [38]:
#set r2_scores to those for the test set for comparision
rf_score = metrics.r2_score(y_test, np.array(rfr.predict((X_test[['y','sy']]))))
p_score = metrics.r2_score(test_df['y'],model.predict(pd.DataFrame(test_df['ds'])).yhat)
p_score, rf_score, comb_score

(0.687577002082496, 0.96882272298543, 0.9178731065886694)

The combine model has a worse?? score in the test set than the RandomForestRegressor. We will now create a class to find optimal hyper-parameters and save the model with.

In [49]:
rfr = RandomForestRegressor()
rf_params = {'n_estimators' : list(range(100, 501, 100)),
                    'criterion' : ['mse','mae'],
                     'max_depth' : [10],
                    'max_features' : ['auto', 'sqrt']}
rscv = RandomizedSearchCV(rfr, rf_params, n_iter=2)
rscv.fit(X_train,y_train)
rfr = rscv.best_estimator_

In [50]:
rf_score = rscv.best_score_

In [51]:
rfr = rscv.best_estimator_

In [52]:
pred_df = combine_predict(prep_df)
metrics.r2_score(y_test, pred_df)

0.5946014280506553

This is sad...

In [53]:
metrics.r2_score(rfr.predict(X_test), y_test)

0.9873837279672111

Clearly, we should just save the optimized Random Forest Regressor at this point and forget about Facebook's Prophet. It was a nice idea, but it didn't pan out in this case.

In [54]:
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(rfr, f)